In [10]:
import re
import requests
from bs4 import BeautifulSoup

def get_kiwix_article(url):
    response = requests.get(url)

    headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0",
    "Referer": "https://anime-world.in/",
    }
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        iframes=soup.find_all('iframe')
        print(iframes)
        # url2 = soup.iframe["src"]
        # html_doc = requests.get(url2, headers=headers).text
        # link = re.search(r'file:".*?(http[^",]+)', html_doc).group(1)
        # print(url2)
        return soup
    else:
        print(f"Failed to fetch article: {response.status_code}")
        return None

def get_first_valid_link(page_source):
    """Fetch the first link in a Wikipedia article's HTML content, ignoring text in parentheses, using a stateful parser."""

    # if data is None:
    #     print("Failed to retrieve article data.")
    #     return None

    # if "parse" not in data:
    #     print("parse not in data")
    #     return None

    # # Extract HTML content from the JSON structure
    # html_content = data["parse"]["text"]["*"]
    soup = get_kiwix_article(page_source)
    # soup = BeautifulSoup(html_content, "html.parser")
    # Check for a redirect message
    redirect_msg = soup.find("div", class_="redirectMsg")
    if redirect_msg:
        # If there's a redirect, find the <a> tag within the .redirectText <li> item
        redirect_link = redirect_msg.find("li").find("a", href=True)
        if redirect_link:
            return redirect_link['href'].split("/wiki/")[1]  # Return the title of the redirect target

    # Read through each paragraph to find first link not in parentheses
    # Initialize a flag to track whether we're inside parentheses
    in_parentheses = False
    # Track the depth of nested parentheses
    parentheses_depth = 0

    for p in soup.find_all("p"):
        # Use .children instead of .descendants to only get direct child nodes
        for child in p.children:
            if isinstance(child, NavigableString):
                # Update the in_parentheses flag based on the presence of parentheses
                for char in child:
                    if char == '(':
                        in_parentheses = True
                        parentheses_depth += 1
                    elif char == ')' and parentheses_depth > 0:
                        parentheses_depth -= 1
                        if parentheses_depth == 0:
                            in_parentheses = False
            elif child.name == "a" and "href" in child.attrs and not in_parentheses:
                link_href = child.attrs["href"]
                # Make sure it's a valid article link and not a reference link or external link
                if link_href.startswith("/wiki/") and not ":" in link_href:
                    # Extract article title from the link
                    link_title = link_href.split("/wiki/")[1]
                    return link_title
    
    return None

# Example usage
host_address = "http://10.0.0.152:80"
# article_title = "William_Henry_Harrison"
# article_url = host_address + '/A/' + article_title
# http://10.0.0.152/viewer#wikipedia_en_top_nopic_2023-10/A/William_Henry_Harrison
# article_url="http://10.0.0.152/viewer#wikipedia_en_all_nopic_2024-02/A/Wikipedia"
# article_url="http://10.0.0.152/viewer#wikipedia_en_all_nopic_2024-02/A/User%3AThe_other_Kiwix_guy/Landing"
article_url="http://10.0.0.152/viewer#wikipedia_en_all_nopic_2024-02/A/Architecture"
print(article_url)
article_soup = get_kiwix_article(article_url)
if article_soup:
    # print(article_soup.prettify()[:1000])  # Print first 500 characters of the HTML
    print(article_soup.prettify())
    print("First link: " + str(get_first_valid_link(article_url)))

http://10.0.0.152/viewer#wikipedia_en_all_nopic_2024-02/A/Architecture
[<iframe id="content_iframe" onload="on_content_load()" referrerpolicy="no-referrer" src="./skin/blank.html?cacheid=6b1fa032" style="border:0px" title="ZIM content" width="100%">
</iframe>]
<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="default-src 'self' data: 'unsafe-inline' 'unsafe-eval';
                   frame-src 'self';
                   object-src 'none';" http-equiv="Content-Security-Policy"/>
  <title>
   ZIM Viewer
  </title>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="./skin/kiwix.css?cacheid=2158fad9" rel="Stylesheet" type="text/css"/>
  <link href="./skin/taskbar.css?cacheid=e014a885" rel="Stylesheet" type="text/css"/>
  <link href="./skin/autoComplete/css/autoComplete.css?cacheid=ef30cd42" rel="Stylesheet" type="text/css"/>
  <script src="./skin/polyfills.js?cacheid=a0e0343d" type="text/javascript">
  </script>
  <script src="./v

In [28]:
from libzim.reader import Archive
# from libzim.search import Query, Searcher
# from libzim.suggestion import SuggestionSearcher
# import libzim
from urllib.request import urlretrieve
import os
# zim_url = "https://github.com/openzim/zim-testing-suite/raw/main/data/nons/small.zim"
# zim_url = "https://download.kiwix.orgp/zim/wikipedia/wikipedia_en_all_nopic_2024-02.zim"
zim_url = "http://ftp.vim.org/kiwix/zim/wikipedia/wikipedia_en_100_nopic_2024-01.zim"
# zim_url = "http://ftp.vim.org/kiwix/zim/wikipedia/wikipedia_en_top_nopic_2023-10.zim"
zim_path = "/Users/mayanktandon/Downloads/" + os.path.basename(zim_url)
if not os.path.isfile(zim_path):
    urlretrieve(zim_url, zim_path)


# zim_path="/Users/mayanktandon/Downloads/wikipedia_en_top_nopic_2023-10.zim"
# zim_path="/Users/mayanktandon/Downloads/example.zim"
# zim_path="/Users/mayanktandon/Downloads/zimfile.zim"
zim = Archive(zim_path)
print(zim)
print(f"Main entry is at {zim.main_entry.get_item().path}")
entry = zim.get_entry_by_path("A/index")
print(f"Entry {entry.title} at {entry.path} is {entry.get_item().size}b.")
print(bytes(entry.get_item().content).decode("UTF-8"))


RuntimeError: Unknown exception

In [7]:
import libzim
import sys

def bytes_to_human_readable(num, suffix='B'):
    for unit in ['', 'K', 'M', 'G', 'T', 'P', 'E', 'Z']:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"



def print_zim_file_info(zim_file_path):
    try:
        # Load the ZIM file
        myarchive = libzim.Archive(zim_file_path)
        
        # Get the number of articles in the ZIM file
        number_of_articles = str(myarchive.entry_count)
        print(f"ZIM File: {zim_file_path}")
        print(f"Main entry is at {myarchive.main_entry.get_item().path}")
        all_items = myarchive.main_entry.get_item()
        print(all_items)

        print(f"Number of articles: {number_of_articles}")
        
        # Print the size of the archive object in memory
        print(f"Size of archive object in memory: {bytes_to_human_readable(sys.getsizeof(myarchive))}")
        print(f"Size of file on disk: {bytes_to_human_readable(myarchive.filesize)}")
    except Exception as e:
        print(f"Failed to load ZIM file {zim_file_path}: {str(e)}")


# Example usage
# zim_file_path = 'path/to/your/file.zim'
# print_zim_file_info(zim_file_path)

myfolder="/Users/mayanktandon/Downloads/"
zimfiles=["wikipedia_en_chemistry_nopic_2024-03.zim","wikipedia_en_baseball_nopic_2024-02.zim","wikipedia_en_physics_nopic_2024-03.zim","wikipedia_en_mathematics_nopic_2024-03.zim","wikipedia_en_top_nopic_2023-10.zim","wikipedia_en_all_nopic_2024-02.zim"]

zimfile=zimfiles[0]
zim_file_path = myfolder + zimfile
print_zim_file_info(zim_file_path)

# for zimfile in zimfiles:
    # zimfile=zimfiles[0]
    # zim_file_path = myfolder + zimfile
    # print_zim_file_info(zim_file_path)






ZIM File: /Users/mayanktandon/Downloads/wikipedia_en_chemistry_nopic_2024-03.zim
Main entry is at A/index
Item(url=A/index, title=Main Page)
Number of articles: 60728
Size of archive object in memory: 48.0B
Size of file on disk: 186.0MB


In [ ]:

import libzim
import sys

myfolder="/Users/mayanktandon/Downloads/"
zimfiles=["wikipedia_en_chemistry_nopic_2024-03.zim","wikipedia_en_baseball_nopic_2024-02.zim","wikipedia_en_physics_nopic_2024-03.zim","wikipedia_en_mathematics_nopic_2024-03.zim","wikipedia_en_top_nopic_2023-10.zim","wikipedia_en_all_nopic_2024-02.zim"]

zimfile=""
